## Data preparation

In [2]:
import numpy as np
import scipy
import os
import sklearn as skl
import pandas as pd
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
#import librosa
#import librosa.display

In [3]:
# Load metadata and features.
#import sys
#reload(sys)
#sys.setdefaultencoding("ISO-8859-1")

tracks = pd.read_csv('/Users/kathyli/tracks.csv', header=None)
echonest = pd.read_csv('echonest.csv', header=None)

IOError: File tracks.csv does not exist

In [2]:
tracks = pd.read_csv('tracks_edited_v2.csv', header=None, encoding='ISO-8859-1')

<font color='red'>**Jupyter Kernel terminated:**</font> This might be caused by running out of memory or hitting a bug in some library (e.g., forking too many processes, trying to access invalid memory, etc.). Consider restarting or upgrading your project or running the relevant code directly in a terminal to track down the cause, as [explained here](https://github.com/sagemathinc/cocalc/wiki/KernelTerminated).

In [8]:
tracks.head()

NameError: name 'tracks' is not defined

In [0]:
# Change column names from "track.1", ... to named columns

# Change duplicate "listens" column to track listens and album listens

In [0]:
# Extract year released from release date variable

# Check number of songs released per year and pick year with maximum

# Merge track metadata with echonest data for chosen year

# Select sensical variables from merged dataset (i.e., some track metadata and named echonest features)

In [0]:
# Add sentiment analysis by title into dataset - compound score of positive / negative sentiment for song title

# Add geographic location (split US into 4 quadrants) as categorical variable; change into dummy variable using one-hot encoding

In [0]:
# For chosen year, check the number of songs released per genre; pick top three genres to use

# Split dataset into three based on top three genres

# OR - add genre as a categorical variable 

# Remove NAs, if needed 

In [0]:
# Make boxplots of variables in genre dataset 

# Standardize variables as needed

# Repeat for other two genres

## Exploratory analysis

In [0]:
# Check for correlation between variables with correlation plot 

# Check for correlation between arist hotness, familiarity, discovery

In [0]:
# Make scatter plots of numerical variables versus outcome (track listens)

# Correlation plots for categorical variables

## Model selection

In [0]:
# Baseline model - use the year with most songs data and include genre as a categorical variable (use the same top 3 genres)

# Convert genre to dummy variables

# Use linear regression with same variables as above (sentiment analysis, track metadata, echonest named variable, location)

# IDEA: we will see that genre is a big predictor of track listens - thus, we train separate models for each genre to dig deeper into why that is

In [0]:
# For 5-folds, split data into training and testing (one of the genres) - will repeat for other two genres (maybe write function to do this)

# Train linear regression model using all variables

# Regularize using L^1 penalty due to large number of features - pick optimal penalty and compare R^2

# Use GridSearchCV with at least 2-fold validation

# Check resulting model on 5 random folds of data 

# Plot coefficients for resulting model 

In [0]:
# Train random forest regression model 

# Regularize using L^1 penalty due to large number of features - pick optimal penalty and compare R^2

# Use GridSearchCV with at least 2-fold validation

# Check resulting model on 5 random folds of data 

# Plot coefficients for resulting model 

In [0]:
# Explore adding other features to model, such as nonlinear features (interaction terms)

In [0]:
# Rock

In [0]:
# Jazz

In [0]:
# Hip hop

In [0]:
# Compare how models are different between different genres - for instance, if certain variables are more or less important for different genres

In [0]:
# Write function that takes in optimal_model, new song audio and necessary info (like genre, track title, etc.) and uses librosa to extract features
# and output a prediction of number of track listens 